# 🚀 SEGY 파일 처리 - 빠른 시작 가이드

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/knocgp/seismic_data_loading/blob/main/quickstart_colab.ipynb)

이 노트북을 Google Colab에서 바로 실행할 수 있습니다!

## 📋 목차
1. 환경 설정
2. 코드 다운로드
3. SEGY 파일 준비
4. 데이터 처리
5. 결과 저장

## 1️⃣ 환경 설정

필요한 라이브러리를 설치합니다.

In [ ]:
# 라이브러리 설치
print("📦 라이브러리 설치 중...")
!pip install segyio numpy matplotlib -q
print("✅ 설치 완료!")

# 기본 import
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

# Colab 환경 확인
try:
    from google.colab import files, drive
    IN_COLAB = True
    print("✅ Google Colab 환경")
except ImportError:
    IN_COLAB = False
    print("ℹ️  로컬 환경")

## 2️⃣ 코드 다운로드

GitHub에서 SEGY 처리 모듈을 다운로드합니다.

In [ ]:
# GitHub 저장소 클론
print("📥 코드 다운로드 중...")

# 이미 존재하면 삭제
if os.path.exists('seismic_data_loading'):
    !rm -rf seismic_data_loading

# 클론
!git clone https://github.com/knocgp/seismic_data_loading.git

# 디렉토리 이동
%cd seismic_data_loading

print("\n✅ 다운로드 완료!")
print("\n📁 다운로드된 파일:")
!ls -lh *.py

In [ ]:
# 모듈 import
from header_loading import SEGYHeaderLoader, load_segy_header
from data_loading import SEGYDataLoader, load_segy_data
from data_divide import SEGYDataDivider, divide_segy_file

print("✅ 모듈 import 완료!")

## 3️⃣ SEGY 파일 준비

SEGY 파일을 준비하는 3가지 방법이 있습니다:
- **방법 A**: Google Drive 마운트 (권장)
- **방법 B**: 파일 직접 업로드
- **방법 C**: URL에서 다운로드

아래에서 하나를 선택하세요!

### 방법 A: Google Drive 사용 (권장) ⭐

In [ ]:
# Google Drive 마운트
if IN_COLAB:
    print("📂 Google Drive 마운트 중...")
    print("(팝업창에서 Google 계정 선택 후 권한 허용)\n")
    drive.mount('/content/drive')
    print("\n✅ Drive 마운트 완료!")
    
    # Drive에 있는 SEGY 파일 찾기
    print("\n🔍 Drive에서 SEGY 파일 검색 중...")
    !find /content/drive/MyDrive -name "*.segy" -o -name "*.sgy" 2>/dev/null | head -10
    
    print("\n💡 위에서 파일을 찾았다면, 아래 셀에서 경로를 입력하세요.")
    print("💡 파일이 없다면, drive.google.com에서 SEGY 파일을 업로드하세요.")
else:
    print("ℹ️  로컬 환경에서는 이 방법을 사용할 수 없습니다.")

In [ ]:
# Drive 파일 경로 입력
# 예: /content/drive/MyDrive/SEGY_Data/your_file.segy

segy_file = "/content/drive/MyDrive/your_file.segy"  # ← 여기를 수정하세요!

# 파일 존재 확인
if os.path.exists(segy_file):
    file_size = os.path.getsize(segy_file) / (1024 * 1024)
    print(f"✅ 파일 찾음: {segy_file}")
    print(f"📊 파일 크기: {file_size:.2f} MB")
else:
    print(f"❌ 파일을 찾을 수 없습니다: {segy_file}")
    print("\n경로를 확인하거나 다른 방법을 사용하세요.")

### 방법 B: 파일 직접 업로드

In [ ]:
# 파일 업로드 (이 방법을 사용하려면 실행)
if IN_COLAB:
    print("📤 SEGY 파일을 선택하세요...")
    print("(파일 선택 창이 나타나면 로컬 컴퓨터에서 파일을 선택)\n")
    
    uploaded = files.upload()
    
    if uploaded:
        segy_file = list(uploaded.keys())[0]
        file_size = os.path.getsize(segy_file) / (1024 * 1024)
        print(f"\n✅ 업로드 완료: {segy_file}")
        print(f"📊 파일 크기: {file_size:.2f} MB")
else:
    segy_file = input("SEGY 파일 경로를 입력하세요: ")
    if os.path.exists(segy_file):
        print(f"✅ 파일 찾음: {segy_file}")
    else:
        print(f"❌ 파일을 찾을 수 없습니다: {segy_file}")

### 방법 C: URL에서 다운로드

In [ ]:
# URL에서 다운로드 (공개 URL이 있는 경우)
# url = "https://example.com/path/to/file.segy"
# !wget -O downloaded.segy "{url}"
# segy_file = "downloaded.segy"

print("💡 URL이 있다면 위 코드의 주석을 해제하고 실행하세요.")

## 4️⃣ 데이터 처리

이제 SEGY 파일을 분석하고 시각화합니다!

### 📖 헤더 분석

In [ ]:
# 헤더 정보 확인
print("📊 SEGY 헤더 분석 중...\n")

with SEGYHeaderLoader(segy_file) as loader:
    # 요약 정보 출력
    loader.print_header_summary()
    
    # 정보 저장
    info = loader.get_file_info()
    
    # Textual Header 출력
    print("\n" + "="*60)
    print("📝 Textual Header (첫 10줄)")
    print("="*60)
    text = loader.load_textual_header()
    print('\n'.join(text.split('\n')[:10]))

# 주요 정보 저장
total_traces = info['total_traces']
samples_per_trace = info['samples_per_trace']
sample_interval_ms = info['sample_interval_ms']

print(f"\n✅ 헤더 분석 완료!")

### 📈 데이터 로드 및 시각화

In [ ]:
# 데이터 로드 (처음 100개 트레이스)
print("📥 데이터 로드 중...\n")

num_traces_to_load = min(100, total_traces)

with SEGYDataLoader(segy_file) as loader:
    # 데이터 로드
    data = loader.load_traces(0, num_traces_to_load)
    time_axis = loader.get_time_axis()
    
    print(f"✅ 로드 완료!")
    print(f"   데이터 형태: {data.shape}")
    print(f"   시간 범위: {time_axis[0]:.2f} ~ {time_axis[-1]:.2f} ms")
    
    # 데이터 통계
    stats = loader.get_data_statistics(data)
    print(f"\n📊 데이터 통계:")
    print(f"   최소값: {stats['min']:.6e}")
    print(f"   최대값: {stats['max']:.6e}")
    print(f"   평균: {stats['mean']:.6e}")
    print(f"   표준편차: {stats['std']:.6e}")

In [ ]:
# Seismic Section 시각화
print("🎨 시각화 생성 중...\n")

plt.figure(figsize=(14, 8))
plt.imshow(data.T, aspect='auto', cmap='seismic',
           extent=[0, num_traces_to_load, time_axis[-1], time_axis[0]],
           vmin=-np.percentile(np.abs(data), 95),
           vmax=np.percentile(np.abs(data), 95))
plt.colorbar(label='Amplitude', shrink=0.8)
plt.xlabel('Trace Number', fontsize=12)
plt.ylabel('Time/Depth (ms)', fontsize=12)
plt.title(f'Seismic Section (First {num_traces_to_load} Traces)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, linestyle='--')
plt.tight_layout()
plt.show()

print("✅ 시각화 완료!")

In [ ]:
# 개별 트레이스 시각화
print("🎨 개별 트레이스 시각화 중...\n")

with SEGYDataLoader(segy_file) as loader:
    # 4개 위치에서 트레이스 선택
    trace_indices = [
        0,
        total_traces // 4,
        total_traces // 2,
        min(total_traces * 3 // 4, total_traces - 1)
    ]
    
    fig, axes = plt.subplots(1, 4, figsize=(16, 6), sharey=True)
    
    for i, trace_idx in enumerate(trace_indices):
        trace_data = loader.load_trace(trace_idx)
        
        axes[i].plot(trace_data, time_axis, 'b-', linewidth=0.5)
        axes[i].fill_betweenx(time_axis, 0, trace_data, 
                             where=(trace_data > 0), color='red', alpha=0.5)
        axes[i].fill_betweenx(time_axis, 0, trace_data, 
                             where=(trace_data < 0), color='blue', alpha=0.5)
        axes[i].set_xlabel('Amplitude')
        axes[i].set_title(f'Trace #{trace_idx}')
        axes[i].grid(True, alpha=0.3)
        axes[i].invert_yaxis()
    
    axes[0].set_ylabel('Time/Depth (ms)')
    plt.tight_layout()
    plt.show()

print("✅ 트레이스 시각화 완료!")

### ✂️ 데이터 분할 (옵션)

In [ ]:
# 데이터 분할 계획
print("✂️ 데이터 분할 계획 생성 중...\n")

traces_per_chunk = 100  # 청크당 트레이스 수
depth_interval_ms = 500.0  # 깊이/시간 간격 (ms)

with SEGYDataDivider(segy_file) as divider:
    # 분할 정보 생성
    chunks = divider.divide_by_grid(traces_per_chunk, depth_interval_ms)
    
    # 정보 출력
    divider.print_division_info(chunks)

print(f"\n✅ 총 {len(chunks)}개 청크로 분할 가능")

In [ ]:
# 첫 번째 청크 시각화
print("🎨 첫 번째 청크 시각화 중...\n")

if len(chunks) > 0:
    with SEGYDataDivider(segy_file) as divider:
        chunk = chunks[0]
        chunk_data = divider.extract_chunk(
            chunk['trace_range'],
            chunk['sample_range']
        )
        
        t_start, t_end = chunk['trace_range']
        time_start, time_end = chunk['time_range_ms']
        
        plt.figure(figsize=(12, 6))
        plt.imshow(chunk_data.T, aspect='auto', cmap='seismic',
                   extent=[t_start, t_end, time_end, time_start],
                   vmin=-np.percentile(np.abs(chunk_data), 95),
                   vmax=np.percentile(np.abs(chunk_data), 95))
        plt.colorbar(label='Amplitude')
        plt.xlabel('Trace Number')
        plt.ylabel('Time/Depth (ms)')
        plt.title(f'Chunk #0 (Traces {t_start}-{t_end}, Time {time_start:.1f}-{time_end:.1f} ms)')
        plt.tight_layout()
        plt.show()
        
        print(f"✅ 청크 시각화 완료!")
        print(f"   청크 크기: {chunk_data.shape}")

## 5️⃣ 결과 저장 (옵션)

처리된 청크를 파일로 저장합니다.

In [ ]:
# 청크 저장 여부 선택
SAVE_CHUNKS = False  # True로 변경하면 저장

if SAVE_CHUNKS:
    print("💾 청크 저장 중...\n")
    
    # 출력 디렉토리 설정
    if IN_COLAB:
        # Drive에 저장
        from datetime import datetime
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_dir = f'/content/drive/MyDrive/SEGY_Chunks_{timestamp}'
    else:
        output_dir = './segy_chunks'
    
    # 청크 저장
    with SEGYDataDivider(segy_file) as divider:
        divider.save_all_chunks(chunks, output_dir, prefix='chunk')
    
    print(f"\n✅ 저장 완료: {output_dir}")
    print(f"   총 {len(chunks)}개 청크 저장됨")
    
    # 저장된 파일 확인
    saved_files = list(Path(output_dir).glob('*.npy'))
    total_size = sum(f.stat().st_size for f in saved_files) / (1024 * 1024)
    print(f"   총 크기: {total_size:.2f} MB")
else:
    print("ℹ️  청크 저장이 비활성화되어 있습니다.")
    print("   저장하려면 위 셀에서 SAVE_CHUNKS = True로 변경하세요.")

## 🎉 완료!

### 📊 처리 요약

In [ ]:
# 최종 요약
print("="*70)
print("🎊 SEGY 파일 처리 완료!")
print("="*70)
print(f"\n📁 파일: {os.path.basename(segy_file)}")
print(f"\n📊 데이터 정보:")
print(f"   • 총 트레이스: {total_traces:,}")
print(f"   • 샘플/트레이스: {samples_per_trace:,}")
print(f"   • 샘플 간격: {sample_interval_ms} ms")
print(f"   • 데이터 형태: ({total_traces}, {samples_per_trace})")
print(f"\n📦 분할 정보:")
print(f"   • 청크당 트레이스: {traces_per_chunk}")
print(f"   • 깊이/시간 간격: {depth_interval_ms} ms")
print(f"   • 총 청크 수: {len(chunks)}")
print(f"\n✅ 모든 처리가 완료되었습니다!")
print("="*70)

print("\n💡 다음 단계:")
print("   1. 다른 범위의 데이터를 시각화해보세요")
print("   2. 청크를 저장하여 나중에 사용하세요")
print("   3. 커스텀 처리 로직을 추가하세요")
print("\n📚 더 자세한 내용은 COLAB_GUIDE.md를 참고하세요!")

## 📚 추가 리소스

- **GitHub 저장소**: [https://github.com/knocgp/seismic_data_loading](https://github.com/knocgp/seismic_data_loading)
- **상세 가이드**: [COLAB_GUIDE.md](https://github.com/knocgp/seismic_data_loading/blob/main/COLAB_GUIDE.md)
- **전체 튜토리얼**: [segy_processing_tutorial.ipynb](https://github.com/knocgp/seismic_data_loading/blob/main/segy_processing_tutorial.ipynb)

### 🔧 커스텀 처리 예제

아래에서 자유롭게 코드를 작성하고 실험해보세요!

In [ ]:
# 여기에 커스텀 코드를 작성하세요!

# 예제: 특정 범위의 데이터 로드
# with SEGYDataLoader(segy_file) as loader:
#     custom_data = loader.load_traces(100, 200)
#     plt.imshow(custom_data.T, aspect='auto', cmap='seismic')
#     plt.show()

print("💡 자유롭게 실험해보세요!")